In [1]:
# le basics
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from time import strftime

# le mon creation
import env
import acquire as a
import prepare as p
import explore as e
from pprint import pprint

# for text digestion
import unicodedata
import re
import json
from bs4 import BeautifulSoup

# nltk: natural language toolkit -> tokenization, stopwords
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

# what's cookin', good lookin'?
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = a.acquire_readmes()

In [3]:
df.head()

,repo,language,readme_contents
0,microsoft/terminal,C++,![terminal-logos](https://user-images.githubus...
1,microsoft/PowerToys,C#,# Microsoft PowerToys\n\n![Hero image for Micr...
2,huggingface/transformers,Python,<!---\nCopyright 2020 The HuggingFace Team. Al...
3,rust-lang/rust,Rust,# The Rust Programming Language\n\nThis is the...
4,mtdvio/every-programmer-should-know,NaN,> *[Join our community](https://metadevelopmen...


In [4]:
df.shape

(683, 3)

In [5]:
df = df.dropna(subset=['language'])

In [6]:
df.shape

(619, 3)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 619 entries, 0 to 682
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             619 non-null    object
 1   language         619 non-null    object
 2   readme_contents  617 non-null    object
dtypes: object(3)
memory usage: 19.3+ KB


In [8]:
df[df.readme_contents.isnull()]

,repo,language,readme_contents
253,harness/drone,Go,NaN
303,gofiber/fiber,Go,NaN


In [9]:
df = df.drop_duplicates()

In [10]:
df.shape

(619, 3)

In [11]:
df.head(20)

,repo,language,readme_contents
0,microsoft/terminal,C++,![terminal-logos](https://user-images.githubus...
1,microsoft/PowerToys,C#,# Microsoft PowerToys\n\n![Hero image for Micr...
2,huggingface/transformers,Python,<!---\nCopyright 2020 The HuggingFace Team. Al...
3,rust-lang/rust,Rust,# The Rust Programming Language\n\nThis is the...
5,opencv/opencv,C++,## OpenCV: Open Source Computer Vision Library...
6,ripienaar/free-for-dev,HTML,# free-for.dev\n\nDevelopers and Open Source a...
7,gohugoio/hugo,Go,"<a href=""https://gohugo.io/""><img src=""https:/..."
8,microsoft/Web-Dev-For-Beginners,JavaScript,[![GitHub license](https://img.shields.io/gith...
9,webpack/webpack,JavaScript,"<div align=""center"">\n <a href=""https://githu..."
10,videojs/video.js,JavaScript,![Video.js logo][logo]\n\n# [Video.js - HTML5 ...


In [12]:
df.language.value_counts()

JavaScript          145
Python               72
TypeScript           61
Go                   59
Java                 56
C++                  32
Objective-C          20
C                    19
Rust                 18
HTML                 17
Swift                15
Jupyter Notebook     13
PHP                  12
Ruby                 10
Shell                 8
C#                    8
Vue                   7
CSS                   7
Kotlin                5
Vim script            3
CoffeeScript          3
Clojure               2
PowerShell            2
Dart                  2
Batchfile             2
Vim Script            2
Markdown              2
SCSS                  2
TeX                   2
Emacs Lisp            1
Less                  1
Svelte                1
Assembly              1
CMake                 1
Haskell               1
Mustache              1
Lua                   1
Astro                 1
Elixir                1
VimL                  1
Jinja                 1
WebAssembly     

In [13]:
df.readme_contents = df.readme_contents.astype('str')
df = p.prep_df_for_nlp(df,'readme_contents',extra_words=[],exclude_words=[])

In [14]:
df.head()

,repo,language,readme_contents,clean,stem,lemmatized
0,microsoft/terminal,Other,![terminal-logos](https://user-images.githubus...,terminallogoshttpsuserimagesgithubusercontentc...,terminallogoshttpsuserimagesgithubusercontentc...,terminallogoshttpsuserimagesgithubusercontentc...
1,microsoft/PowerToys,Other,# Microsoft PowerToys\n\n![Hero image for Micr...,microsoft powertoys hero image microsoft power...,microsoft powertoy hero imag microsoft powerto...,microsoft powertoys hero image microsoft power...
2,huggingface/transformers,Python,<!---\nCopyright 2020 The HuggingFace Team. Al...,copyright 2020 huggingface team rights reserve...,copyright 2020 huggingfac team right reserv li...,copyright 2020 huggingface team right reserved...
3,rust-lang/rust,Other,# The Rust Programming Language\n\nThis is the...,rust programming language main source code rep...,rust program languag main sourc code repositor...,rust programming language main source code rep...
5,opencv/opencv,Other,## OpenCV: Open Source Computer Vision Library...,opencv open source computer vision library res...,opencv open sourc comput vision librari resour...,opencv open source computer vision library res...


In [15]:
df.shape

(619, 6)

# Explore based on Ryan's notebooks

In [16]:
df.head()

,repo,language,readme_contents,clean,stem,lemmatized
0,microsoft/terminal,Other,![terminal-logos](https://user-images.githubus...,terminallogoshttpsuserimagesgithubusercontentc...,terminallogoshttpsuserimagesgithubusercontentc...,terminallogoshttpsuserimagesgithubusercontentc...
1,microsoft/PowerToys,Other,# Microsoft PowerToys\n\n![Hero image for Micr...,microsoft powertoys hero image microsoft power...,microsoft powertoy hero imag microsoft powerto...,microsoft powertoys hero image microsoft power...
2,huggingface/transformers,Python,<!---\nCopyright 2020 The HuggingFace Team. Al...,copyright 2020 huggingface team rights reserve...,copyright 2020 huggingfac team right reserv li...,copyright 2020 huggingface team right reserved...
3,rust-lang/rust,Other,# The Rust Programming Language\n\nThis is the...,rust programming language main source code rep...,rust program languag main sourc code repositor...,rust programming language main source code rep...
5,opencv/opencv,Other,## OpenCV: Open Source Computer Vision Library...,opencv open source computer vision library res...,opencv open sourc comput vision librari resour...,opencv open source computer vision library res...


# Top five languages

### make the following cell Code to generate language series

JavaScript_words_series = (' '.join(df[df.language == 'JavaScript']['readme_contents']))
Python_words_series = (' '.join(df[df.language == 'Python']['readme_contents']))
TypeScript_words_series = (' '.join(df[df.language == 'TypeScript']['readme_contents']))
Go_words_words_series = (' '.join(df[df.language == 'Go']['readme_contents']))
Java_words_series = (' '.join(df[df.language == 'Java']['readme_contents']))

all_words_series = (' '.join(df['readme_contents']))


In [18]:
df.language.value_counts()

Other         226
JavaScript    145
Python         72
TypeScript     61
Go             59
Java           56
Name: language, dtype: int64

In [23]:
len(df.readme_contents.groupby(df.lemmatized).value_counts())

618

In [24]:
df.shape

(619, 6)